# Shopify Sync 

A minimal guide for synchronizing **Shopify** customers and products with a **SQLite** database.

## 1. Create Schema
- **Function**: `create_schema(db_path="shopify_data.db")`
- **Purpose**: Sets up tables (customers, products, sync_history, etc.).
- **Run** once initially or whenever you reset the database.

## 2. Sync Customers
- **Function**: `fetch_and_store_customers(db_path="shopify_data.db", threshold_days=90)`
- **Full Sync** if no prior sync or if last sync is older than `threshold_days`.
- **Incremental Sync** if within `threshold_days`.

## 3. Sync Products
- **Function**: `fetch_and_store_products(db_path="shopify_data.db", threshold_days=90)`
- Same logic as customers (full or incremental).

## 4. Usage Example
```python
create_schema("shopify_data.db")
fetch_and_store_customers()
fetch_and_store_products()

In [22]:
import requests
import json
import os
import keyring
import sqlite3
import datetime

In [23]:
# API Configuration
api_key = keyring.get_password("Shopify", "SHOPIFY_API_KEY")
access_token = keyring.get_password("Shopify", "SHOPIFY_API_PASSWORD")
shop_url = keyring.get_password("Shopify", "SHOPIFY_SHOP_URL")
graphql_endpoint = 'https://' + shop_url + '/admin/api/2025-01/graphql.json'

In [24]:
headers = {
    'X-Shopify-Access-Token': access_token,
    'Content-Type': 'application/json'
}


In [47]:
def create_schema(db_path="shopify_data.db"):
    """
    Creates (if not exists) the tables for storing:
    - Shopify customers
    - Shopify products
    - Sync history (to track last sync date for each type)
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # ------------------
    # Customer tables
    # ------------------
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS customers (
            id TEXT PRIMARY KEY,
            email TEXT,
            firstName TEXT,
            lastName TEXT,
            createdAt TEXT,
            displayName TEXT,
            phone TEXT,
            lifetimeDuration TEXT,
            note TEXT,
            numberOfOrders INTEGER,
            updatedAt TEXT,
            verifiedEmail BOOLEAN,
            amountSpent REAL
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS customer_addresses (
            address_id TEXT PRIMARY KEY,
            customer_id TEXT,
            address1 TEXT,
            address2 TEXT,
            city TEXT,
            province TEXT,
            country TEXT,
            zip TEXT,
            is_default BOOLEAN,
            company TEXT,
            phone TEXT,
            FOREIGN KEY(customer_id) REFERENCES customers(id)
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS customer_tags (
            customer_id TEXT,
            tag TEXT,
            FOREIGN KEY(customer_id) REFERENCES customers(id)
        )
    ''')
    
    # ------------------
    # Product tables
    # ------------------
    # Make sure to include columns for tracksInventory, status, and SEO fields
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS products (
            id TEXT PRIMARY KEY,
            title TEXT,
            handle TEXT,
            description TEXT,
            productType TEXT,
            vendor TEXT,
            createdAt TEXT,
            updatedAt TEXT,
            publishedAt TEXT,
            totalInventory INTEGER,
            tracksInventory BOOLEAN,
            status TEXT,
            seo_title TEXT,
            seo_description TEXT
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS product_tags (
            product_id TEXT,
            tag TEXT,
            FOREIGN KEY(product_id) REFERENCES products(id)
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS product_variants (
            variant_id TEXT PRIMARY KEY,
            product_id TEXT,
            title TEXT,
            sku TEXT,
            price REAL,
            inventoryQuantity INTEGER,
            createdAt TEXT,
            updatedAt TEXT,
            FOREIGN KEY(product_id) REFERENCES products(id)
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS product_images (
            image_id TEXT PRIMARY KEY,
            product_id TEXT,
            altText TEXT,
            originalSrc TEXT,
            FOREIGN KEY(product_id) REFERENCES products(id)
        )
    ''')
    
    # ------------------
    # Sync history
    # ------------------
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS sync_history (
            data_type TEXT PRIMARY KEY,
            last_sync_date TEXT
        )
    ''')
    
    conn.commit()
    conn.close()

In [26]:
def log_sync_event(data_type: str, db_path="shopify_data.db"):
    """
    Records (or updates) the time of the last sync for a specific data type.
    This function sets the sync date to 'now' in UTC.
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    now_utc = datetime.datetime.now(datetime.timezone.utc).isoformat()
    cursor.execute('''
        INSERT OR REPLACE INTO sync_history (data_type, last_sync_date)
        VALUES (?, ?)
    ''', (data_type, now_utc))
    
    conn.commit()
    conn.close()

In [27]:
def get_last_sync_date(data_type: str, db_path="shopify_data.db"):
    """
    Returns a datetime object for the last time the given data_type was synced.
    If there's no record, returns None.
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    cursor.execute('''
        SELECT last_sync_date FROM sync_history
        WHERE data_type = ?
    ''', (data_type,))
    
    row = cursor.fetchone()
    conn.close()
    
    if row and row[0]:
        return datetime.datetime.fromisoformat(row[0])
    return None

In [28]:
def should_full_sync(data_type: str, threshold_days: int = 90, db_path="shopify_data.db"):
    """
    Determines if a full sync is required for data_type.
    - If last sync is older than threshold_days, return True.
    - If there's no sync date, also return True.
    - Otherwise, return False (we can do incremental).
    """
    last_sync = get_last_sync_date(data_type, db_path=db_path)
    if not last_sync:
        # No sync record, so a full sync is needed
        return True
    
    now_utc = datetime.datetime.now(datetime.timezone.utc)
    delta = now_utc - last_sync
    return delta.days > threshold_days

In [29]:
customer_fields = '''
    id
    email
    firstName
    lastName
    createdAt
    displayName
    phone
    lifetimeDuration
    note
    numberOfOrders
    tags
    updatedAt
    verifiedEmail
    addresses {
        address1
        address2
        city
        province
        country
        zip
    }
    amountSpent {
        amount
    }
    defaultAddress {
        address1
        address2
        city
        company
        id
        phone
        zip
    }
'''

In [48]:
product_fields = '''
    id
    title
    handle
    description
    productType
    vendor
    createdAt
    updatedAt
    publishedAt
    totalInventory
    tags
    tracksInventory
    status
    seo {
        title
        description
    }
    variants(first: 20) {
      edges {
        node {
          id
          title
          sku
          price
          inventoryQuantity
          createdAt
          updatedAt
        }
      }
    }
    images(first: 20) {
      edges {
        node {
          id
          altText
          originalSrc
        }
      }
    }
'''

In [31]:
def fetch_all_customers():
    """
    Fetches all customers, paging through the entire result set.
    """
    customers = []
    has_next_page = True
    cursor_var = None
    
    while has_next_page:
        if cursor_var:
            query = f'''
            {{
              customers(first: 50, after: "{cursor_var}") {{
                pageInfo {{
                  hasNextPage
                  endCursor
                }}
                edges {{
                  node {{
                    {customer_fields}
                  }}
                }}
              }}
            }}
            '''
        else:
            query = f'''
            {{
              customers(first: 50) {{
                pageInfo {{
                  hasNextPage
                  endCursor
                }}
                edges {{
                  node {{
                    {customer_fields}
                  }}
                }}
              }}
            }}
            '''
            
        try:
            response = requests.post(graphql_endpoint, json={'query': query}, headers=headers)
            response.raise_for_status()
            result = response.json()
            
            page_customers = result['data']['customers']['edges']
            for edge in page_customers:
                customers.append(edge['node'])
            
            has_next_page = result['data']['customers']['pageInfo']['hasNextPage']
            cursor_var = result['data']['customers']['pageInfo']['endCursor']
            
        except Exception as e:
            print(f"Error fetching customers: {e}")
            break
    
    return customers

In [32]:
def fetch_customers_since(since_date: datetime.datetime):
    """
    Fetch customers created or updated after 'since_date'.
    """
    customers = []
    has_next_page = True
    cursor_var = None
    
    # Convert date to string in ISO8601
    date_str = since_date.isoformat()
    
    while has_next_page:
        if cursor_var:
            query = f'''
            {{
              customers(
                first: 50, 
                after: "{cursor_var}", 
                query: "createdAt > \\"{date_str}\\" OR updatedAt > \\"{date_str}\\""
              ) {{
                pageInfo {{
                  hasNextPage
                  endCursor
                }}
                edges {{
                  node {{
                    {customer_fields}
                  }}
                }}
              }}
            }}
            '''
        else:
            query = f'''
            {{
              customers(
                first: 50, 
                query: "createdAt > \\"{date_str}\\" OR updatedAt > \\"{date_str}\\""
              ) {{
                pageInfo {{
                  hasNextPage
                  endCursor
                }}
                edges {{
                  node {{
                    {customer_fields}
                  }}
                }}
              }}
            }}
            '''
            
        try:
            response = requests.post(graphql_endpoint, json={'query': query}, headers=headers)
            response.raise_for_status()
            result = response.json()
            
            if 'errors' in result:
                print(f"GraphQL errors: {result['errors']}")
                break
            
            page_customers = result['data']['customers']['edges']
            for edge in page_customers:
                customers.append(edge['node'])
            
            has_next_page = result['data']['customers']['pageInfo']['hasNextPage']
            cursor_var = result['data']['customers']['pageInfo']['endCursor']
            
        except Exception as e:
            print(f"Error fetching customers: {e}")
            break
    
    return customers

In [33]:
def store_customers_in_db(customers, db_path="shopify_data.db"):
    """
    Inserts or updates the customer data into the SQLite database.
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    for cust in customers:
        customer_id = cust.get('id', '')
        email = cust.get('email')
        first_name = cust.get('firstName')
        last_name = cust.get('lastName')
        created_at = cust.get('createdAt')
        display_name = cust.get('displayName')
        phone = cust.get('phone')
        lifetime_duration = cust.get('lifetimeDuration')
        note = cust.get('note')
        number_of_orders = cust.get('numberOfOrders', 0)
        tags_list = cust.get('tags', [])
        updated_at = cust.get('updatedAt')
        verified_email = cust.get('verifiedEmail', False)
        
        # amountSpent is a dict with { 'amount': ... }
        amount_spent = cust.get('amountSpent', {}).get('amount', 0.0)
        
        # Insert or replace the customer record
        cursor.execute('''
            INSERT OR REPLACE INTO customers (
                id, email, firstName, lastName, createdAt,
                displayName, phone, lifetimeDuration, note,
                numberOfOrders, updatedAt, verifiedEmail, amountSpent
            )
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            customer_id, email, first_name, last_name, created_at,
            display_name, phone, lifetime_duration, note,
            number_of_orders, updated_at, verified_email, amount_spent
        ))
        
        # Handle tags
        cursor.execute('DELETE FROM customer_tags WHERE customer_id = ?', (customer_id,))
        for tag in tags_list:
            cursor.execute('''
                INSERT INTO customer_tags (customer_id, tag)
                VALUES (?, ?)
            ''', (customer_id, tag))
        
        # Insert addresses
        cursor.execute('DELETE FROM customer_addresses WHERE customer_id = ?', (customer_id,))
        
        addresses = cust.get('addresses', [])
        default_address = cust.get('defaultAddress')
        default_addr_id = default_address['id'] if default_address else None
        
        for addr in addresses:
            is_default = False
            address_id = None
            if default_address and (
                addr.get('address1') == default_address.get('address1') and
                addr.get('address2') == default_address.get('address2') and
                addr.get('city') == default_address.get('city') and
                addr.get('zip') == default_address.get('zip') and
                addr.get('phone') == default_address.get('phone')
            ):
                is_default = True
                address_id = default_addr_id
            else:
                address_id = f"{customer_id}-addr-{addresses.index(addr)}"
            
            cursor.execute('''
                INSERT OR REPLACE INTO customer_addresses (
                    address_id, customer_id, address1, address2, city, province,
                    country, zip, is_default, company, phone
                )
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                address_id,
                customer_id,
                addr.get('address1'),
                addr.get('address2'),
                addr.get('city'),
                addr.get('province'),
                addr.get('country'),
                addr.get('zip'),
                int(is_default),
                default_address.get('company') if is_default and default_address else None,
                addr.get('phone')
            ))
    
    conn.commit()
    conn.close()

In [34]:
def fetch_and_store_customers(db_path="shopify_data.db", threshold_days=90):
    """
    Main entry point to fetch and store customers based on last sync date.
    - If last sync was more than threshold_days (default 90 days) ago (or never synced),
      then do a full fetch.
    - Otherwise, do an incremental fetch since the last sync date.
    - Finally, update the sync history.
    """
    data_type = "customers"
    
    if should_full_sync(data_type, threshold_days=threshold_days, db_path=db_path):
        print("Performing FULL sync for customers.")
        customers = fetch_all_customers()
    else:
        last_sync = get_last_sync_date(data_type, db_path=db_path)
        # Provide a fallback: if for some reason the date is missing, do full
        if not last_sync:
            print("No last sync date found, performing FULL sync for customers.")
            customers = fetch_all_customers()
        else:
            print(f"Performing INCREMENTAL sync for customers since {last_sync.isoformat()}")
            customers = fetch_customers_since(last_sync)
    
    store_customers_in_db(customers, db_path=db_path)
    log_sync_event(data_type, db_path=db_path)
    print(f"Synced {len(customers)} customer records.")

In [35]:
def fetch_all_products():
    """
    Fetches all products, paging through the entire result set.
    """
    products = []
    has_next_page = True
    cursor_var = None
    
    while has_next_page:
        if cursor_var:
            query = f'''
            {{
              products(first: 50, after: "{cursor_var}") {{
                pageInfo {{
                  hasNextPage
                  endCursor
                }}
                edges {{
                  node {{
                    {product_fields}
                  }}
                }}
              }}
            }}
            '''
        else:
            query = f'''
            {{
              products(first: 50) {{
                pageInfo {{
                  hasNextPage
                  endCursor
                }}
                edges {{
                  node {{
                    {product_fields}
                  }}
                }}
              }}
            }}
            '''
            
        try:
            response = requests.post(graphql_endpoint, json={'query': query}, headers=headers)
            response.raise_for_status()
            result = response.json()
            
            # Check for errors
            if 'errors' in result:
                print(f"GraphQL errors: {result['errors']}")
                break
            
            page_products = result['data']['products']['edges']
            for edge in page_products:
                products.append(edge['node'])
            
            has_next_page = result['data']['products']['pageInfo']['hasNextPage']
            cursor_var = result['data']['products']['pageInfo']['endCursor']
            
        except Exception as e:
            print(f"Error fetching products: {e}")
            break
    
    return products

In [36]:
def fetch_products_since(since_date: datetime.datetime):
    """
    Fetch products created or updated after 'since_date'.
    Uses the "createdAt" or "updatedAt" fields in GraphQL.
    """
    products = []
    has_next_page = True
    cursor_var = None
    
    date_str = since_date.isoformat()
    
    while has_next_page:
        if cursor_var:
            query = f'''
            {{
              products(
                first: 50,
                after: "{cursor_var}",
                query: "createdAt > \\"{date_str}\\" OR updatedAt > \\"{date_str}\\""
              ) {{
                pageInfo {{
                  hasNextPage
                  endCursor
                }}
                edges {{
                  node {{
                    {product_fields}
                  }}
                }}
              }}
            }}
            '''
        else:
            query = f'''
            {{
              products(
                first: 50,
                query: "createdAt > \\"{date_str}\\" OR updatedAt > \\"{date_str}\\""
              ) {{
                pageInfo {{
                  hasNextPage
                  endCursor
                }}
                edges {{
                  node {{
                    {product_fields}
                  }}
                }}
              }}
            }}
            '''
            
        try:
            response = requests.post(graphql_endpoint, json={'query': query}, headers=headers)
            response.raise_for_status()
            result = response.json()
            
            # Check for errors
            if 'errors' in result:
                print(f"GraphQL errors: {result['errors']}")
                break
            
            page_products = result['data']['products']['edges']
            for edge in page_products:
                products.append(edge['node'])
            
            has_next_page = result['data']['products']['pageInfo']['hasNextPage']
            cursor_var = result['data']['products']['pageInfo']['endCursor']
            
        except Exception as e:
            print(f"Error fetching products: {e}")
            break
    
    return products

In [49]:
def store_products_in_db(products, db_path="shopify_data.db"):
    """
    Inserts or updates the product data into the SQLite database.
    Includes fields for tracksInventory, status, seo.title, and seo.description.
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    for prod in products:
        product_id = prod.get('id')
        title = prod.get('title')
        handle = prod.get('handle')
        description = prod.get('description')
        product_type = prod.get('productType')
        vendor = prod.get('vendor')
        created_at = prod.get('createdAt')
        updated_at = prod.get('updatedAt')
        published_at = prod.get('publishedAt')
        total_inventory = prod.get('totalInventory', 0)
        
        # NEW FIELDS
        tracks_inventory = prod.get('tracksInventory', False)
        status = prod.get('status')
        seo_info = prod.get('seo', {})
        seo_title = seo_info.get('title')
        seo_description = seo_info.get('description')
        
        tags_list = prod.get('tags', [])
        variants_info = prod.get('variants', {}).get('edges', [])
        images_info = prod.get('images', {}).get('edges', [])
        
        # Insert or replace the product record
        cursor.execute('''
            INSERT OR REPLACE INTO products (
                id, title, handle, description, productType, vendor, 
                createdAt, updatedAt, publishedAt, totalInventory,
                tracksInventory, status, seo_title, seo_description
            )
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            product_id,
            title,
            handle,
            description,
            product_type,
            vendor,
            created_at,
            updated_at,
            published_at,
            total_inventory,
            int(tracks_inventory),
            status,
            seo_title,
            seo_description
        ))
        
        # Handle product tags
        cursor.execute('DELETE FROM product_tags WHERE product_id = ?', (product_id,))
        for tag in tags_list:
            cursor.execute('''
                INSERT INTO product_tags (product_id, tag)
                VALUES (?, ?)
            ''', (product_id, tag))
        
        # Handle product variants
        cursor.execute('DELETE FROM product_variants WHERE product_id = ?', (product_id,))
        for variant_edge in variants_info:
            variant = variant_edge['node']
            variant_id = variant.get('id')
            variant_title = variant.get('title')
            variant_sku = variant.get('sku')
            variant_price = variant.get('price')
            variant_inventory = variant.get('inventoryQuantity', 0)
            variant_created_at = variant.get('createdAt')
            variant_updated_at = variant.get('updatedAt')
            
            cursor.execute('''
                INSERT OR REPLACE INTO product_variants (
                    variant_id, product_id, title, sku, price, 
                    inventoryQuantity, createdAt, updatedAt
                )
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                variant_id,
                product_id,
                variant_title,
                variant_sku,
                variant_price,
                variant_inventory,
                variant_created_at,
                variant_updated_at
            ))
        
        # Handle product images
        cursor.execute('DELETE FROM product_images WHERE product_id = ?', (product_id,))
        for image_edge in images_info:
            image_node = image_edge['node']
            image_id = image_node.get('id')
            alt_text = image_node.get('altText')
            original_src = image_node.get('originalSrc')
            
            cursor.execute('''
                INSERT OR REPLACE INTO product_images (
                    image_id, product_id, altText, originalSrc
                )
                VALUES (?, ?, ?, ?)
            ''', (
                image_id,
                product_id,
                alt_text,
                original_src
            ))
    
    conn.commit()
    conn.close()

In [38]:
def fetch_and_store_products(db_path="shopify_data.db", threshold_days=90):
    """
    Main entry point to fetch and store products based on last sync date.
    - If last sync was more than threshold_days (default 90) ago (or never synced),
      then do a full fetch.
    - Otherwise, do an incremental fetch since the last sync date.
    - Finally, update the sync history.
    """
    data_type = "products"
    
    if should_full_sync(data_type, threshold_days=threshold_days, db_path=db_path):
        print("Performing FULL sync for products.")
        products = fetch_all_products()
    else:
        last_sync = get_last_sync_date(data_type, db_path=db_path)
        if not last_sync:
            print("No last sync date found, performing FULL sync for products.")
            products = fetch_all_products()
        else:
            print(f"Performing INCREMENTAL sync for products since {last_sync.isoformat()}")
            products = fetch_products_since(last_sync)
    
    store_products_in_db(products, db_path=db_path)
    log_sync_event(data_type, db_path=db_path)
    print(f"Synced {len(products)} product records.")

In [52]:
create_schema('shopify_data.db')

In [50]:
fetch_and_store_customers()


Performing INCREMENTAL sync for customers since 2025-01-10T02:39:15.437823+00:00
Synced 0 customer records.


In [54]:
fetch_and_store_products()

Performing INCREMENTAL sync for products since 2025-01-10T03:03:53.797695+00:00
Synced 0 product records.


## Need to convert the above into the export format for nbdev so can run as python script and build a gui